# Test processing of a single calendar aging data

The calendar aging data will include RPTs separated by rests.

2/10/2023

In [6]:
import os, sys
from matplotlib import pyplot as plt

if os.path.basename(os.getcwd()) == 'notebooks-2023':
    os.chdir('../')
    sys.path.insert(0, 'src/')
    
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib ipympl

import src.plotter as plotter
import src.vas as vas
import src.parsers as parsers

%load_ext autoreload
%autoreload 2

plotter.initialize(plt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Initialize the helper class
vh = vas.VasHelper()

Initializing Voltaiq Analytic Studio Helper...
Initializing test records...
Initializing devices...
Done.


In [31]:
device_name = 'UMBL2022FEB_CELL152032'
cal = parsers.CyclingDataParser(device_name, vh,
                               key='CAL_4P1V_P45C_5P0PSI',
                               offs=-1)

Working on "UMBL2022FEB_CELL152032_CAL_4P1V_P45C_5P0PSI_20220804_R1"...
Initialization complete.


In [33]:
rpt = parsers.RptDataParser(cal.df_rpt)

# First inspect the entire dataset

In [35]:
plot_vars = ['voltage_v', 'cycle_index', 'step_index']
xvar = 'test_time_s'
nrows = len(plot_vars)

fh, axs = plt.subplots(nrows=nrows, ncols=1, 
                       figsize=(8, nrows * 3), sharex=True)

for yvar, ax in zip(plot_vars, axs):
    ax.plot(cal.df[xvar], cal.df[yvar])
    ax.set_ylabel(yvar)
    if ax != axs[-1]:
        ax.tick_params(labelbottom=False)
    
axs[-1].set_xlabel(xvar);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Process the RPT Data

In [37]:
df_by_cyc = cal.get_cycling_info()

In [38]:
df_by_cyc

,charge_capacity_ah,charge_energy_wh,discharge_capacity_ah,discharge_energy_wh,tot_charge_capacity_ah,tot_charge_energy_wh,tot_discharge_capacity_ah,tot_discharge_energy_wh,charge_cc_time_s,charge_cv_time_s,charge_rest_delta_voltage_v,discharge_cc_time_s,discharge_rest_delta_voltage_v
cycle_index,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,0.000000,0.000000,0.166989,0.529444,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.081929,0.268156,0.173947,0.549583,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,0.163842,0.548961,0.180890,0.571412,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0.245768,0.836077,0.187848,0.594215,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,0.327682,1.124840,0.194805,0.617347,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,NaN,NaN,NaN,NaN,65.510401,248.427301,63.082264,230.448515,NaN,NaN,NaN,NaN,NaN
433,NaN,NaN,NaN,NaN,65.585376,248.739998,63.089209,230.475301,NaN,NaN,NaN,NaN,NaN
434,NaN,NaN,NaN,NaN,65.806319,249.667386,63.096160,230.502344,NaN,NaN,NaN,NaN,NaN


In [60]:
rpt_start_cycle_list = rpt.get_rpt_start_cycles(-1)
df_by_rpt = rpt.get_rpt_info(rpt_start_cycle_list, offs=-1)

rpt_start_cycle_list

# Append the throughput data by matching on the cycling info DataFrame
rpt_tot_discharge_capacity_ah = []
rpt_tot_test_time_days = []

for cycle_index in df_by_rpt['cycle_index']:
    rpt_tot_discharge_capacity_ah.append(df_by_cyc['tot_discharge_capacity_ah'][df_by_cyc.index == cycle_index].values[0])
    rpt_tot_test_time_days.append( np.max(cal.df['test_time_s'][cal.df['cycle_index'] == cycle_index]) / 3600 / 24)
    
df_by_rpt['tot_discharge_capacity_ah'] = rpt_tot_discharge_capacity_ah
df_by_rpt['tot_test_time_days'] = rpt_tot_test_time_days

df_by_rpt

,cycle_index,c20_discharge_capacity_ah,c20_charge_capacity_ah,deltav_4p2v_12hr,deltav_3p0v_2p5hr,dvdq_data,tot_discharge_capacity_ah,tot_test_time_days
0,1,2.434416,2.445500,-0.009750,1.056454,"{'dch_q': [1.2734200014701109e-06, 0.000348479...",0.166989,0.126769
1,36,2.397702,2.406825,-0.008938,0.019988,"{'dch_q': [-2.232168358854987e-06, 0.000345173...",5.164741,9.175795
2,69,2.371701,2.380212,-0.008613,0.018201,"{'dch_q': [3.7645809371867714e-08, 0.000347098...",10.083436,18.168444
3,102,2.344042,2.349607,-0.008125,0.200858,"{'dch_q': [1.1531298747870835e-06, 0.000348536...",17.189702,29.703526
4,134,2.311314,2.315817,-0.007800,0.199233,"{'dch_q': [1.1696192944411725e-07, 0.000347331...",21.979904,42.875630
5,166,2.297220,2.292856,-0.008287,0.197770,"{'dch_q': [1.0863017621817575e-06, 0.000348938...",26.711662,56.110409
6,198,2.263411,2.267380,-0.008613,0.277398,"{'dch_q': [2.2272595522814668e-07, 0.000347742...",31.404567,70.532423
7,229,2.245935,2.249312,-0.008450,0.278049,"{'dch_q': [2.2721047074225475e-06, 0.000349970...",36.038464,83.463599
8,260,2.226712,2.229944,-0.008938,0.303562,"{'dch_q': [-3.3897377564835263e-06, 0.00034432...",40.633379,96.346834
9,290,2.212401,2.212141,-0.008938,0.306000,"{'dch_q': [-8.83986201032985e-07, 0.0003472050...",45.181205,109.192294


In [61]:
fh2, axs2 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)
fh3, axs3 = plt.subplots(nrows=2, ncols=1, figsize=(10, 2 * 4), sharex=True)

colors = list(reversed(plt.cm.jet(np.linspace(0,1,len(df_by_rpt)+1))))

i = 0
for cycle_index, row in df_by_rpt.iterrows():
    
    data = row['dvdq_data'];
    
    dch_dict = dict()
    dch_dict['dch_q'] = data['dch_q']
    dch_dict['dch_v'] = data['dch_v']
    dch_dict['dch_dvdq'] = data['dch_dvdq']
    
    chg_dict = dict()
    chg_dict['chg_q'] = data['chg_q']
    chg_dict['chg_v'] = data['chg_v']
    chg_dict['chg_dvdq'] = data['chg_dvdq']
    
    dch_df = pd.DataFrame(dch_dict)
    dch_df.to_csv(f'{device_name}_c20_data_day{row["tot_test_time_days"]:.1f}_dch.csv')

    chg_df = pd.DataFrame(chg_dict)
    chg_df.to_csv(f'{device_name}_c20_data_day{row["tot_test_time_days"]:.1f}_chg.csv')
    
    axs2[0].plot(data['chg_q'], data['chg_v'], c=colors[i])
    axs2[1].plot(data['dch_q'], data['dch_v'], c=colors[i])
    
    axs3[0].plot(data['chg_q'], data['chg_dvdq'], c=colors[i])
    axs3[1].plot(data['dch_q'], data['dch_dvdq'], c=colors[i])

    axs3[0].set_ylim([0, 0.75])
    axs3[1].set_ylim([-0.75, 0])
                 
    i += 1
    
axs2[0].set_title(device_name)
axs2[1].set_xlabel('Capacity (Ah)')
axs2[0].set_ylabel('Voltage (V)')
axs2[1].set_ylabel('Voltage (V)')
axs3[0].set_title(device_name)
axs3[1].set_xlabel('Capacity (Ah)')
axs3[0].set_ylabel('dV/dQ (V/Ah)')
axs3[1].set_ylabel('dV/dQ (V/Ah)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plotting Cycling and RPT data together

In [42]:
plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['c20_discharge_capacity_ah'], marker='o', label='C/20 DCH')
plt.plot(df_by_cyc['tot_discharge_capacity_ah']/2.5, df_by_cyc['discharge_capacity_ah'], label='1C DCH')
plt.legend()
plt.ylabel('Capacity (Ah)')
plt.xlabel('Throughput (Ah/Ah)')
plt.grid(False)
plt.title(device_name);

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_3p0v_2p5hr'], marker='o', label='$\Delta$V, 3.0V, 2.5hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah/Ah)')
plt.grid(False)
plt.title(device_name);

plt.figure(figsize=(10, 7))
plt.plot(df_by_rpt['tot_discharge_capacity_ah']/2.5, df_by_rpt['deltav_4p2v_12hr'], marker='o', label='$\Delta$V, 4.2V, 12hr')
plt.legend()
plt.ylabel('$\Delta$ V')
plt.xlabel('Throughput (Ah/Ah)')
plt.grid(False)
plt.title(device_name);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Process HPPC data from each RPT

In [44]:
df_hppc = rpt.get_hppc_info(rpt_start_cycle_list, -1)

# Append the throughput data by matching on the cycling info DataFrame
hppc_tot_discharge_capacity_ah = []

for cycle_index in df_hppc['cycle_index']:
    hppc_tot_discharge_capacity_ah.append(df_by_cyc['tot_discharge_capacity_ah'][df_by_cyc.index == cycle_index].values[0])
    
df_hppc['tot_discharge_capacity_ah'] = hppc_tot_discharge_capacity_ah

df_hppc

,cycle_index,resistance_discharge_ohms,resistance_charge_ohms,capacity_ah,tot_discharge_capacity_ah
0,1,0.085297,0.070920,0.000000,0.166989
1,1,0.065611,0.058914,0.074961,0.166989
2,1,0.057160,0.053502,0.149931,0.166989
3,1,0.050527,0.048900,0.224906,0.166989
4,1,0.048364,0.047210,0.299866,0.166989
...,...,...,...,...,...
389,408,0.080968,0.078675,1.574404,62.915513
390,408,0.080568,0.078092,1.649377,62.915513
391,408,0.079984,0.076141,1.724341,62.915513
392,408,0.079591,0.069129,1.799317,62.915513


In [45]:
plt.figure()

colors = list(reversed(plt.cm.jet(np.linspace(0,1,len(df_hppc['cycle_index'].unique())+1))))

for idx, cycle_index in enumerate(reversed(df_hppc['cycle_index'].unique())):
    
    curr_df = df_hppc[df_hppc['cycle_index'] == cycle_index]
    curr_throughput_ah = curr_df['tot_discharge_capacity_ah'].head(1).values[0]
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_discharge_ohms'], label=f'{curr_throughput_ah:.0f} Ah', marker='o', color=colors[idx])
    plt.plot(curr_df['capacity_ah'], curr_df['resistance_charge_ohms'], label=f'', marker='x', linestyle='', color=colors[idx])
       
plt.legend()
plt.xlabel('Capacity (Ah)')
plt.ylabel('10s Resistance ($\Omega$)')
plt.title(device_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'UMBL2022FEB_CELL152032')